In [10]:
import os
import requests
import json
from dotenv import load_dotenv
load_dotenv()

API_URL = os.environ.get('API_BASE_URL', 'http://localhost:8000')
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', '')
PINECONE_INDEX = os.environ.get('PINECONE_INDEX', '')
PINECONE_HOST = os.environ.get('PINECONE_HOST', '')

print("API_URL:", API_URL)
# print("PINECONE_API_KEY:", PINECONE_API_KEY)
print("PINECONE_INDEX:", PINECONE_INDEX)
print("PINECONE_HOST:", PINECONE_HOST)

API_URL: http://localhost:8000
PINECONE_INDEX: simonas-serverless-1536
PINECONE_HOST: https://simonas-serverless-1536-75c816a.svc.apw5-4e34-81fa.pinecone.io


In [28]:
# Ingest a file
url = f"{API_URL}/api/v1/ingest"

payload = {
    "files": [
        {
            "name": "chunking",
            "url": "https://arxiv.org/pdf/2402.05131.pdf"
        }
    ],
    "vector_database": {
        "type": "pinecone",
        "config": {
            "api_key": PINECONE_API_KEY,
            "host": PINECONE_HOST,
        }
    },
    "chunk_config": {
        "partition_strategy": "auto", # For tables use "hi_res"
        "split_method": "semantic", # or 'by_title'
        "min_chunk_tokens": 50,
        "max_token_size": 300,
        "rolling_window_size": 1
    },
    "index_name": PINECONE_INDEX,
    "encoder": {
        "name": "text-embedding-ada-002",
        "provider": "openai",
        "dimensions": 1536
    },
}

response = requests.post(url, json=payload)

print(response.json())

{'success': True, 'index_name': 'simonas-serverless-1536'}


In [29]:
# Query the index
query_url = f"{API_URL}/api/v1/query"

query_payload = {
    "input": "What are the chunking strategies?",
    "vector_database": {
        "type": "pinecone",
        "config": {
            "api_key": PINECONE_API_KEY,
            "host": PINECONE_HOST,
        }
    },
    "index_name": PINECONE_INDEX,
    "encoder": {
        "name": "text-embedding-ada-002",
        "provider": "openai",
        "dimensions": 1536
    },
}

query_response = requests.post(query_url, json=query_payload)

# NOTE: Filter out fields before given to LLM
# Include title, content, source, page_number, chunk_index
formatted_json = json.dumps(query_response.json(), indent=4)
print(formatted_json)

{
    "success": true,
    "data": [
        {
            "id": "19baf12d-cc24-44a7-bbb8-b3068e2217e3",
            "doc_url": "https://arxiv.org/pdf/2402.05131.pdf",
            "document_id": "doc_a4f19222-92af-40ae-ae82-bf7aa0f29bd7",
            "content": "Results in table 5 show that element-based chunking strategies o\ufb00er the best question-answering accuracy, which is consistent with page retrieval and para- graph retrieval accuracy. Lastly, our approach stands out for its e\ufb03ciency. Not only is element-based chunking generalizable without the need to select the chunk size, but when com- pared to the aggregation results that yield the highest retrieval scores. Element- based chunking achieves the highest retrieval scores with only half the number of chunks required compared to methods that do not consider the structure of the documents (62,529 v.s. 112,155). This can reduce the indexing cost and im- prove query latency because there are only half as many vectors to inde

In [ ]:
# Ingest a file
url = f"{API_URL}/api/v1/ingest"

payload = {
    "files": [
        {
            "type": "PDF",
            "url": "https://arxiv.org/pdf/2402.05131.pdf"
        }
    ],
    "vector_database": {
        "type": "weaviate",
        "config": {
            "api_key": "9eXH8oNR0uqN3GvvzAgaUD11ltPnGqZG2RFQ",
            "host": "https://superagent-ragas-1575sjfq.weaviate.network"
        }
    },
    "index_name": "homanp11",
    "encoder": "cohere",
    "webhook_url": "https://webhook.site/0e217d1c-49f1-424a-9992-497db09f7793"
}

response = requests.post(url, json=payload)

print(response.json())

In [ ]:
# Query the index
query_url = f"{API_URL}/api/v1/query"

query_payload = {
    "input": "What are the chunking strategies?",
    "vector_database": {
        "type": "weaviate",
        "config": {
            "api_key": "9eXH8oNR0uqN3GvvzAgaUD11ltPnGqZG2RFQ",
            "host": "https://superagent-ragas-1575sjfq.weaviate.network"
        }
    },
    "index_name": "homanp11",
    "encoder": "cohere",
}

query_response = requests.post(query_url, json=query_payload)

print(query_response.json())

In [ ]:
data = query_response.json().get('data', [])
data

In [ ]:
data = query_response.json().get('data', [])
data

In [14]:
# Delete the index
query_url = f"{API_URL}/api/v1/delete"

delete_payload = {
    "file_url": "https://arxiv.org/pdf/2210.03629.pdf",
    "vector_database": {
        "type": "pinecone",
        "config": {
            "api_key": PINECONE_API_KEY,
            "host": PINECONE_HOST,
        }
    },
    "index_name": PINECONE_INDEX,
    "encoder": "openai",
}

delete_response = requests.delete(query_url, json=delete_payload)

print(delete_response.json())

{'detail': [{'type': 'missing', 'loc': ['body', 'files'], 'msg': 'Field required', 'input': {'file_url': 'https://arxiv.org/pdf/2210.03629.pdf', 'vector_database': {'type': 'pinecone', 'config': {'api_key': 'f4adc79e-ad40-4426-a78a-9878e2ed4a79', 'host': 'https://simonas-serverless-1536-75c816a.svc.apw5-4e34-81fa.pinecone.io'}}, 'index_name': 'simonas-serverless-1536', 'encoder': 'openai'}, 'url': 'https://errors.pydantic.dev/2.6/v/missing'}, {'type': 'model_attributes_type', 'loc': ['body', 'encoder'], 'msg': 'Input should be a valid dictionary or object to extract fields from', 'input': 'openai', 'url': 'https://errors.pydantic.dev/2.6/v/model_attributes_type'}]}


In [ ]:
# Delete the index
query_url = f"{API_URL}/api/v1/delete"

delete_payload = {
    "file_url": "https://arxiv.org/pdf/2402.05131.pdf",
    "vector_database": {
        "type": "weaviate",
        "config": {
            "api_key": "9eXH8oNR0uqN3GvvzAgaUD11ltPnGqZG2RFQ",
            "host": "https://superagent-ragas-1575sjfq.weaviate.network"
        }
    },
    "index_name": "homanp11",
    "encoder": "cohere"
}

delete_response = requests.delete(query_url, json=delete_payload)

print(delete_response.json())